# Create Heat Map for significant Spliceosome genes

This notebook looks at the significant genes in at least one cancer in the Spliceosome pathway. Pancancer heat maps are created with circle size showing significance and color showing differences in median.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as pu

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [2]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
sig_df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

prot_list = list(sig_df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')
prot_enr.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Spliceosome Homo sapiens hsa03040,88/134,2.936551e-44,8.604095e-42,0,0,4.860965,487.246193,RBM25;EIF4A3;DDX42;HNRNPU;PRPF19;USP39;EFTUD2;...,KEGG_2016
1,RNA transport Homo sapiens hsa03013,77/172,1.113872e-23,1.631823e-21,0,0,3.313652,175.131871,CYFIP2;NUP107;NUP188;RPP30;EIF4A3;PHAX;PNN;XPO...,KEGG_2016
2,DNA replication Homo sapiens hsa03030,23/36,3.613010e-12,3.528707e-10,0,0,4.729007,124.592696,RFC5;FEN1;RFC3;PCNA;RFC4;MCM7;RFC1;RFC2;PRIM1;...,KEGG_2016
3,mRNA surveillance pathway Homo sapiens hsa03015,38/91,2.801846e-11,2.052353e-09,0,0,3.090914,75.103503,SMG1;RBM8A;RNMT;CSTF3;EIF4A3;MSI1;PPP2R2A;MSI2...,KEGG_2016
4,Mismatch repair Homo sapiens hsa03430,15/23,1.473843e-08,8.636717e-07,0,0,4.827342,87.050531,RFC5;RFC3;PCNA;RFC4;RFC1;RFC2;RPA1;MLH1;POLD3;...,KEGG_2016


# Step 2: Get the list of significant genes in the Spliceosome pathway

In [3]:
splice = prot_enr.res2d.Genes[0]
genes = splice.split(';')
print('total genes:',len(genes))

total genes: 88


# Step 3: Create HeatMap

Slice out about 15 genes from the significant genes in the pathway. 15 genes show up well when the plotCircleHeatMap function is called. Change the variable representing a different list of genes to visualize all the genes in the pathway.  

In [4]:
genes_1 = genes[0:16]
genes_2 = genes[16:32]
genes_3 = genes[32:47]
genes_4 = genes[47:62]
genes_5 = genes[62:77]
genes_6 = genes[77:]

In [5]:
bool_df = sig_df.Proteomics.isin(genes_1)
plot_df = sig_df[bool_df]
plot_df.head()

,Proteomics,P_Value,Medians,Cancer
126,USP39,0.000017,0.419766,Gbm
322,EFTUD2,0.000127,0.304429,Gbm
446,CTNNBL1,0.000245,0.224219,Gbm
487,RBM25,0.000294,0.439854,Gbm
488,PRPF19,0.000299,0.226303,Gbm


In [17]:
pu.plotCircleHeatMap(plot_df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height= 500)

Loading BokehJS ...

# Step 4: Create a HeatMap with both pos and neg differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [10]:
pos_neg_df = pd.read_csv(root+R'\pos_neg_df.csv')

In [11]:
pos_neg_df['Cancer'].unique()

array(['Gbm', 'Hnscc', 'Luad', 'Lscc', 'Brca', 'Ov', 'En', 'Colon'],
      dtype=object)

In [12]:
get = pos_neg_df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
len(genes_pn.Proteomics.unique())

9

In [16]:
pu.plotCircleHeatMap(genes_pn, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=500, plot_width=600)

Loading BokehJS ...